# 主成分分析 演習問題 - 解答・解説

このNotebookは、講義で提示された演習問題に対する解答と解説を記載したものです。\n\n- **問1**: 主成分分析の実行と寄与率の確認\n- **問2**: 主成分の解釈と顧客の可視化

## 0. 準備：ライブラリのインポート

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# グラフのスタイル設定
sns.set_style('whitegrid')
plt.rcParams['font.family'] = 'Hiragino Sans'
plt.rcParams['figure.figsize'] = [10, 6]

---

## 問1：主成分分析の実行と寄与率の確認

### 1-1. データの読み込みと確認

In [ ]:
# データの読み込み
df = pd.read_csv('sample_data.csv')

# データの先頭5行を表示
df.head()

In [ ]:
# データの基本情報を確認
df.info()

### 1-2. 分析対象のデータの選択と標準化

In [ ]:
# customer_id列を除外して、分析対象のデータを準備
X = df.drop('customer_id', axis=1)

# 標準化の実行
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 標準化後のデータを確認（平均が0、分散が1に近くなっている）
print('標準化後の平均:', X_scaled.mean(axis=0)) 
print('標準化後の分散:', X_scaled.var(axis=0))

### 1-3 & 1-4. 主成分分析の実行と寄与率の計算

In [ ]:
# 主成分分析の実行 (n_componentsを指定しないと、変数の数だけ主成分が作成される)
pca = PCA()
pca.fit(X_scaled)

# 寄与率を計算
explained_variance_ratio = pca.explained_variance_ratio_

# 累積寄与率を計算
cumulative_explained_variance_ratio = np.cumsum(explained_variance_ratio)

# 結果をDataFrameで表示
pca_result_df = pd.DataFrame({
    'PC': [f'PC{i+1}' for i in range(len(explained_variance_ratio))],
    'Contribution Ratio': explained_variance_ratio,
    'Cumulative Contribution Ratio': cumulative_explained_variance_ratio
})

pca_result_df

### 1-5. スクリープロットの作成

In [ ]:
fig, ax1 = plt.subplots(figsize=(12, 7))

# 棒グラフ（寄与率）
sns.barplot(x=pca_result_df['PC'], y=pca_result_df['Contribution Ratio'], color=sns.color_palette()[0], ax=ax1, label='寄与率')
ax1.set_ylabel('寄与率')
ax1.set_ylim(0, 1)

# 折れ線グラフ（累積寄与率）
ax2 = ax1.twinx()
sns.lineplot(x=pca_result_df['PC'], y=pca_result_df['Cumulative Contribution Ratio'], color=sns.color_palette()[1], marker='o', ax=ax2, label='累積寄与率')
ax2.set_ylabel('累積寄与率')
ax2.set_ylim(0, 1.05)

# 80%のラインを追加
ax2.axhline(0.8, ls='--', color='red', label='80%ライン')

plt.title('スクリープロット')
fig.legend(loc='upper right', bbox_to_anchor=(0.9, 0.9))
plt.show()

### 1-6. 解答

上記の表とグラフから、累積寄与率は第2主成分(PC2)の時点で約85%となり、80%の基準を超えていることがわかります。\n
したがって、**第2主成分まで採用すれば、元のデータが持つ情報の約85%を説明できる**と言えます。

---

## 問2：主成分の解釈と顧客の可視化

問1の結果から、今回は第2主成分までを採用して分析を進めます。

### 2-1 & 2-2. 主成分負荷量の算出と可視化

In [ ]:
# 主成分負荷量を計算
# 主成分負荷量 = 固有ベクトル * (主成分の標準偏差の平方根)
# scikit-learnのPCAでは、主成分の分散は`explained_variance_`に格納されている

loadings = pca.components_.T * np.sqrt(pca.explained_variance_)

# DataFrameに変換
loadings_df = pd.DataFrame(loadings, index=X.columns, columns=[f'PC{i+1}' for i in range(len(X.columns))])

# 第1, 第2主成分の負荷量をヒートマップで可視化
plt.figure(figsize=(8, 6))
sns.heatmap(loadings_df[['PC1', 'PC2']], annot=True, cmap='coolwarm', fmt='.2f')
plt.title('主成分負荷量 (PC1 & PC2)')
plt.show()

### 2-3. 解答：各主成分の意味の考察

ヒートマップを見ると、各主成分と元の変数の相関関係がわかります。

- **第1主成分 (PC1)**:
  - `quality`, `design`, `support`, `store_cleanliness`, `purchase_amount`, `frequency` の値が**正に大きい**（赤色が濃い）。
  - 一方で `price` の値が**負に大きい**（青色が濃い）。
  - これは、「品質やデザイン、サポートが良く、店舗も綺麗で、たくさん購入してくれる顧客」ほどPC1のスコアが高くなり、「価格満足度が高い（＝価格に敏感な）顧客」ほどスコアが低くなる傾向を示唆しています。
  - したがって、**PC1は「総合的なロイヤルティ（顧客の質）」軸**と解釈できます。右に行くほど優良顧客、左に行くほど価格重視の顧客となります。

- **第2主成分 (PC2)**:
  - `price` の値が**正に大きい**。
  - `purchase_amount`, `frequency` の値も正だが、`price`ほどではない。
  - `quality`, `design`, `support`, `store_cleanliness` の値は負になっている。
  - PC2のスコアが高い顧客は、「価格満足度が高い」が、「品質やデザインなどの付加価値」はあまり重視していない層と考えられます。
  - したがって、**PC2は「価格志向 vs 品質・付加価値志向」軸**と解釈できます。上に行くほど価格を重視し、下に行くほど品質などを重視する顧客となります。

### 2-4 & 2-5. 主成分スコアの算出と可視化

In [ ]:
# n_components=2で再度PCAを実行し、主成分スコアを計算
pca = PCA(n_components=2)
score = pca.fit_transform(X_scaled)

# DataFrameに変換
score_df = pd.DataFrame(score, columns=['PC1', 'PC2'])

# 散布図で可視化
plt.figure(figsize=(12, 8))
sns.scatterplot(x='PC1', y='PC2', data=score_df)
plt.title('主成分スコアの散布図')
plt.xlabel('PC1: 総合的なロイヤルティ軸')
plt.ylabel('PC2: 価格志向 vs 品質志向 軸')

# 軸を追加
plt.axhline(0, color='grey', linestyle='--', linewidth=0.8)
plt.axvline(0, color='grey', linestyle='--', linewidth=0.8)

plt.show()

### 2-6. 解答：顧客分布の考察

作成した散布図から、顧客をいくつかのグループに分けて特徴を考えることができます。

- **右下のクラスター (PC1: 正, PC2: 負)**
  - 「総合的なロイヤルティ」が高く、「品質・付加価値」を重視する層。
  - いわゆる**「優良顧客層」**や**「ファン顧客」**と言えるグループです。高価格帯でも品質やサポートを理由に購入してくれる可能性が高いです。

- **左上のクラスター (PC1: 負, PC2: 正)**
  - 「総合的なロイヤルティ」が低く、「価格」を強く重視する層。
  - **「価格重視層」**や**「セールハンター」**と言えるグループです。割引やキャンペーンに強く反応する可能性があります。

- **中央付近のクラスター**
  - 特定の嗜好が強くなく、バランスの取れた顧客層。
  - **「一般顧客層」**と考えられます。

このように、主成分分析を用いることで、多角的なアンケート結果を2つの軸に集約し、顧客のセグメンテーション（グループ分け）や特徴把握に繋げることができます。